Install necessary packages

In [ ]:
!pip install transformers sentence-transformers tensorflow

In [ ]:
!pip install streamlit pyngrok

In [ ]:
!pip install firebase-admin

In [ ]:
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


Uploading Firebase Realtime Database Credentials

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving pawsense-ai-firebase-adminsdk-fbsvc-a7956019c9.json to pawsense-ai-firebase-adminsdk-fbsvc-a7956019c9.json


Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the Model

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/fine_tuned_flan_t5_v4last")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_flan_t5_v4last")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/fine_tuned_flan_t5_v4last.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ChatBot Functionality and Frontend

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import tensorflow as tf
from gtts import gTTS
import io
import firebase_admin
from firebase_admin import credentials, db

# ---------- Firebase Setup ----------
cred = credentials.Certificate("pawsense-ai-firebase-adminsdk-fbsvc-a7956019c9.json")
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://pawsense-ai-default-rtdb.firebaseio.com/'
    })

# ---------- Page Settings ----------
st.set_page_config(page_title="🐾 PawSense", layout="wide")

# ---------- Load Models ----------
model_name = "/content/drive/MyDrive/fine_tuned_flan_t5_v4last"

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model()

embedder = SentenceTransformer('all-MiniLM-L6-v2')

@st.cache_data
def load_dataset_from_firebase():
    ref = db.reference("qa_pairs")
    data = ref.get()

    dataset = []
    corpus = []

    # If data is a list
    if isinstance(data, list):
        items = data
    # If data is a dict with keys like "0", "1", etc.
    elif isinstance(data, dict):
        items = list(data.values())
    else:
        items = []

    for item in items:
        if item is not None:
            input_text = item["input"].replace("**", "")
            output_text = item["output"].replace("**", "")
            dataset.append({"input": input_text, "output": output_text})
            corpus.append(input_text)

    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    return dataset, corpus_embeddings

dataset, corpus_embeddings = load_dataset_from_firebase()

# ---------- Custom Styling ----------
st.markdown("""<style>
body { background-color: #121212; color: white; }
.dashboard-title { font-size: 36px; font-weight: bold; color: #fff; margin-bottom: 10px; }
.description { font-size: 16px; color: #d3d3d3; margin-bottom: 30px; }
.card { background-color: #2b2b2b; padding: 30px; border-radius: 15px; box-shadow: 0 4px 12px rgba(0,0,0,0.5); transition: all 0.3s ease-in-out; cursor: pointer; text-align: center; text-decoration: none; display: block; }
.card:hover { transform: translateY(-5px); box-shadow: 0 6px 18px rgba(0,0,0,0.6); }
.card-title { font-size: 24px; font-weight: 600; margin-bottom: 10px; color: white; }
.card-desc { font-size: 14px; color: #cfcfcf; }
.sidebar-title { font-weight: bold; font-size: 20px; margin-top: 20px; }
a { color: inherit; }
</style>""", unsafe_allow_html=True)

# ---------- Sidebar Navigation ----------
with st.sidebar:
    st.markdown("<div class='sidebar-title'>PawSense Navigation</div>", unsafe_allow_html=True)
    service = st.selectbox("Choose a Chatbot:", ["Home", "Pet Care", "Cattle Care"])
    st.button("Logout", key="logout", help="Click to logout")

# ---------- Page Routing ----------
if service == "Home":
    st.markdown('<div class="dashboard-title">🐾 Welcome to PawSense!</div>', unsafe_allow_html=True)
    st.markdown('<div class="description">Your AI-powered assistant for pet and cattle care. Choose a chatbot from the sidebar to begin.</div>', unsafe_allow_html=True)

    col1, col2 = st.columns(2)

    with col1:
        pet_card = f"""<a href="?service=Pet Care">
        <div class="card"><div class="card-title">Pet Care 🐶</div>
        <div class="card-desc">Ask questions related to your pet’s health, nutrition, and behavior.</div></div></a>"""
        st.markdown(pet_card, unsafe_allow_html=True)

    with col2:
        cattle_card = f"""<a href="?service=Cattle Care">
        <div class="card"><div class="card-title">Cattle Care 🐄</div>
        <div class="card-desc">Support for dairy, poultry, and other cattle-related queries.</div></div></a>"""
        st.markdown(cattle_card, unsafe_allow_html=True)

# ---------- Pet Care Chatbot ----------
elif service == "Pet Care":
    st.markdown('<div class="dashboard-title">🐶 Pet Care Chatbot</div>', unsafe_allow_html=True)
    if "pet_chat_history" not in st.session_state:
        st.session_state.pet_chat_history = []

    pet_question = st.text_input("Ask your Pet Care question:")

    if st.button("Get Answer"):
        if pet_question.strip():
            with st.spinner("Thinking..."):
                question_embedding = embedder.encode(pet_question, convert_to_tensor=True)
                hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=3)[0]
                prompt = "You are an expert in pet care.\n\n"
                for hit in hits:
                    q = dataset[hit['corpus_id']]["input"]
                    a = dataset[hit['corpus_id']]["output"]
                    prompt += f"Q: {q}\nA: {a}\n\n"
                prompt += f"Q: {pet_question}\nA (detailed):"
                input_ids = tokenizer(prompt, return_tensors="tf", padding=True, truncation=True).input_ids
                attention_mask = tokenizer(prompt, return_tensors="tf", padding=True, truncation=True).attention_mask
                output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=256, num_beams=4)
                answer = tokenizer.decode(output[0], skip_special_tokens=True)
                st.session_state.pet_chat_history.append((pet_question, answer))

    for q, a in reversed(st.session_state.pet_chat_history):
        st.markdown(f"**You 🙋‍♀️** {q}")
        st.markdown(f"**PawSense 🐾** {a}")
        tts = gTTS(text=a, lang='en')
        fp = io.BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        with st.expander("🔊 Hear this answer"):
            st.audio(fp.read(), format='audio/mp3')
        st.markdown("---")

# ---------- Cattle Care Chatbot ----------
elif service == "Cattle Care":
    st.markdown('<div class="dashboard-title">🐄 Cattle Care Chatbot</div>', unsafe_allow_html=True)
    if "cattle_chat_history" not in st.session_state:
        st.session_state.cattle_chat_history = []

    cattle_question = st.text_input("Ask your Cattle Care question:")

    if st.button("Get Answer"):
        if cattle_question.strip():
            with st.spinner("Thinking..."):
                question_embedding = embedder.encode(cattle_question, convert_to_tensor=True)
                hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=3)[0]
                prompt = "You are an expert in cattle care.\n\n"
                for hit in hits:
                    q = dataset[hit['corpus_id']]["input"]
                    a = dataset[hit['corpus_id']]["output"]
                    prompt += f"Q: {q}\nA: {a}\n\n"
                prompt += f"Q: {cattle_question}\nA (detailed):"
                input_ids = tokenizer(prompt, return_tensors="tf", padding=True, truncation=True).input_ids
                attention_mask = tokenizer(prompt, return_tensors="tf", padding=True, truncation=True).attention_mask
                output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=256, num_beams=4)
                answer = tokenizer.decode(output[0], skip_special_tokens=True)
                st.session_state.cattle_chat_history.append((cattle_question, answer))

    for q, a in reversed(st.session_state.cattle_chat_history):
        st.markdown(f"**You 🙋‍♀️** {q}")
        st.markdown(f"**PawSense 🐾** {a}")
        tts = gTTS(text=a, lang='en')
        fp = io.BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        with st.expander("🔊 Hear this answer"):
            st.audio(fp.read(), format='audio/mp3')
        st.markdown("---")


Writing app.py


Run app.py

In [ ]:
!streamlit run app.py &>/dev/null&

To get the Public URL

In [ ]:
from pyngrok import conf

conf.get_default().auth_token =  "2qYswpecIJhKSdstSAKprD4SsCf_6hP15b2RiSoHcXKtMdv6c"

from pyngrok import ngrok

tunnel = ngrok.connect(8501, "http")
public_url = tunnel.public_url

print(f"Public URL: {public_url}")

Public URL: https://729e31e6deba.ngrok-free.app


To check the files inside the model folder

In [ ]:
!ls /content/drive/MyDrive/fine_tuned_flan_t5_v4last

config.json		 spiece.model		tokenizer.json
generation_config.json	 tf_model.h5
special_tokens_map.json  tokenizer_config.json


To Disconnect the tunnels if it exceeds more than 3

In [ ]:
from pyngrok import ngrok

# Get a list of all active tunnels
tunnels = ngrok.get_tunnels()

for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)  # Disconnect using the public URL
    print(f"Disconnected tunnel: {tunnel.public_url}")

print("All active ngrok tunnels have been disconnected.")

Disconnected tunnel: https://b4af-34-53-31-12.ngrok-free.app
Disconnected tunnel: https://9d66-34-53-31-12.ngrok-free.app
Disconnected tunnel: https://ef75-34-53-31-12.ngrok-free.app
All active ngrok tunnels have been disconnected.
